In [13]:
import sqlite3
import pandas as pd
from datetime import datetime, timedelta

# Kết nối SQLite
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Tạo bảng student
cursor.execute('''
CREATE TABLE student (
    student_id INTEGER PRIMARY KEY,
    name TEXT,
    class TEXT,
    course_id INTEGER,
    score REAL
)
''')

# Chèn dữ liệu vào bảng student
student_data = [
    (1, 'Nguyen Minh Hoang', 'May Tinh', 12, 6.7),
    (2, 'Tran Thi Lan', 'Kinh Te', 34, 9.2),
    (3, 'Pham Van Nam', 'Toan Tin', None, 7.9),
    (4, 'Le Thanh Huyen', 'Toan Tin', 20, 7.2),
    (5, 'Vu Quoc Anh', 'May Tinh', 24, 8.0),
    (6, 'Dang Thuy Linh', 'May Tinh', 24, 5.5),
    (7, 'Bui Tien Dung', 'Kinh Te', 34, 9.2),
    (8, 'Ho Ngoc Mai', 'Toan Tin', 20, 8.8),
    (9, 'Duong Huu Phuc', 'Kinh Te', None, 7.2),
    (10, 'Cao Thi Hanh', 'May Tinh', None, 7.0)
]
cursor.executemany("INSERT INTO student VALUES (?, ?, ?, ?, ?)", student_data)

# Tạo bảng course
cursor.execute('''
CREATE TABLE course (
    id INTEGER PRIMARY KEY,
    course_name TEXT
)
''')

# Chèn dữ liệu vào bảng course
course_data = [
    (12, 'Giai tich'),
    (34, 'Thong ke'),
    (26, 'Tin hoc'),
    (20, 'Xac suat')
]
cursor.executemany("INSERT INTO course VALUES (?, ?)", course_data)

conn.commit()

# 1. Kết nối hai bảng
# Tích Decartes
print("Tích Decartes:")
df = pd.read_sql("SELECT * FROM student CROSS JOIN course", conn)
print(df.head())

# INNER JOIN
print("\nINNER JOIN:")
df = pd.read_sql("""
SELECT student.*, course.course_name FROM student
INNER JOIN course ON student.course_id = course.id
""", conn)
print(df.head())

# LEFT JOIN
print("\nLEFT JOIN:")
df = pd.read_sql("""
SELECT student.*, course.course_name FROM student
LEFT JOIN course ON student.course_id = course.id
""", conn)
print(df.head())

# RIGHT JOIN bằng cách đổi vị trí bảng (SQLite không hỗ trợ RIGHT JOIN trực tiếp)
print("\nRIGHT JOIN:")
df = pd.read_sql("""
SELECT course.*, student.name, student.class, student.score FROM course
LEFT JOIN student ON student.course_id = course.id
""", conn)
print(df.head())

# FULL OUTER JOIN bằng cách UNION của LEFT và RIGHT JOIN
print("\nFULL OUTER JOIN:")
df = pd.read_sql("""
SELECT student.*, course.course_name FROM student
LEFT JOIN course ON student.course_id = course.id
UNION
SELECT student.*, course.course_name FROM student
RIGHT JOIN course ON student.course_id = course.id
""", conn)
print(df.head())

Tích Decartes:
   student_id               name     class  course_id  score  id course_name
0           1  Nguyen Minh Hoang  May Tinh       12.0    6.7  12   Giai tich
1           1  Nguyen Minh Hoang  May Tinh       12.0    6.7  20    Xac suat
2           1  Nguyen Minh Hoang  May Tinh       12.0    6.7  26     Tin hoc
3           1  Nguyen Minh Hoang  May Tinh       12.0    6.7  34    Thong ke
4           2       Tran Thi Lan   Kinh Te       34.0    9.2  12   Giai tich

INNER JOIN:
   student_id               name     class  course_id  score course_name
0           1  Nguyen Minh Hoang  May Tinh         12    6.7   Giai tich
1           2       Tran Thi Lan   Kinh Te         34    9.2    Thong ke
2           4     Le Thanh Huyen  Toan Tin         20    7.2    Xac suat
3           7      Bui Tien Dung   Kinh Te         34    9.2    Thong ke
4           8        Ho Ngoc Mai  Toan Tin         20    8.8    Xac suat

LEFT JOIN:
   student_id               name     class  course_id  score

In [14]:
# 2a
cursor.execute("""
UPDATE student
SET course_id = (SELECT id FROM course WHERE course_name = 'Tin hoc')
WHERE course_id IS NULL
""")

cursor.execute("DELETE FROM student WHERE course_id NOT IN (SELECT id FROM course)")
conn.commit()

print("\nTổng số sinh viên, điểm trung bình theo lớp:")
df = pd.read_sql("""
SELECT class, COUNT(*) as total_students, AVG(score) as avg_score
FROM student
GROUP BY class
""", conn)
print(df)

#2b2b. Tổng số sinh viên, điểm trung bình theo môn học
print("\nTổng số sinh viên, điểm trung bình theo môn học:")
df = pd.read_sql("""
SELECT course.course_name, COUNT(student.student_id) as total_students, AVG(student.score) as avg_score
FROM student
INNER JOIN course ON student.course_id = course.id
GROUP BY course.course_name
""", conn)
print(df)

#2c2c. Phân loại thi đua theo điểm trung bình
print("\nPhân loại thi đua:")
df = pd.read_sql("""
SELECT course.course_name, AVG(student.score) as avg_score,
CASE
    WHEN AVG(student.score) >= 9.0 THEN 'Xuất sắc'
    WHEN AVG(student.score) BETWEEN 6.0 AND 8.9 THEN 'Tốt'
    ELSE 'Kém'
END as classification
FROM student
INNER JOIN course ON student.course_id = course.id
GROUP BY course.course_name
""", conn)
print(df)


Tổng số sinh viên, điểm trung bình theo lớp:
      class  total_students  avg_score
0   Kinh Te               3   8.533333
1  May Tinh               2   6.850000
2  Toan Tin               3   7.966667

Tổng số sinh viên, điểm trung bình theo môn học:
  course_name  total_students  avg_score
0   Giai tich               1   6.700000
1    Thong ke               2   9.200000
2     Tin hoc               3   7.366667
3    Xac suat               2   8.000000

Phân loại thi đua:
  course_name  avg_score classification
0   Giai tich   6.700000            Tốt
1    Thong ke   9.200000       Xuất sắc
2     Tin hoc   7.366667            Tốt
3    Xac suat   8.000000            Tốt


In [15]:
#3. Xếp hạng sinh viên theo điểm số
print("\nXếp hạng theo điểm số:")
df = pd.read_sql("""
SELECT student_id, name, class, score,
RANK() OVER (ORDER BY score DESC) as rank
FROM student
""", conn)
print(df)

#4. Thêm cột graduation_date
cursor.execute("ALTER TABLE student ADD COLUMN graduation_date DATETIME")
cursor.execute("""
UPDATE student
SET graduation_date = datetime('now', '+' || CAST((10 - score) AS INTEGER) || ' months')
""")
conn.commit()

# Hiển thị dữ liệu sau khi cập nhật graduation_date
print("\nDữ liệu sau khi thêm graduation_date:")
df = pd.read_sql("SELECT * FROM student", conn)
print(df)

# Đóng kết nối
conn.close()



Xếp hạng theo điểm số:
   student_id               name     class  score  rank
0           2       Tran Thi Lan   Kinh Te    9.2     1
1           7      Bui Tien Dung   Kinh Te    9.2     1
2           8        Ho Ngoc Mai  Toan Tin    8.8     3
3           3       Pham Van Nam  Toan Tin    7.9     4
4           4     Le Thanh Huyen  Toan Tin    7.2     5
5           9     Duong Huu Phuc   Kinh Te    7.2     5
6          10       Cao Thi Hanh  May Tinh    7.0     7
7           1  Nguyen Minh Hoang  May Tinh    6.7     8

Dữ liệu sau khi thêm graduation_date:
   student_id               name     class  course_id  score  \
0           1  Nguyen Minh Hoang  May Tinh         12    6.7   
1           2       Tran Thi Lan   Kinh Te         34    9.2   
2           3       Pham Van Nam  Toan Tin         26    7.9   
3           4     Le Thanh Huyen  Toan Tin         20    7.2   
4           7      Bui Tien Dung   Kinh Te         34    9.2   
5           8        Ho Ngoc Mai  Toan Tin       